## AO3文章下载器

和大家之前说过，相关代码我会放出来，现在代码已经上传了去Github搜索 czw90130/AO3_DataAnalyze，我也和大家讲解一下实现方法，希望能激发大家学习编程兴趣。

这里先讲解爬虫是怎么实现的，至于文本分析器。下周有空再写吧，实在是肝不动了！

先说一下我的配置环境：
* 操作系统：Ubuntu 18.04
* Python环境：conda 4.8.2 Python 3.7.6
* 编辑器使用的是 Jupyter Notebook 代码也是ipynb格式。想学python或者数据分析的可以用这个，适合组织文章。

使用到的 Python 库有：
* sys os time 不解释了
* re 正则表达式解析
* tqdm 进度条
* selenuim 自动化测试/爬虫
* BeautifulSoup html 标签解析器

文章分析器使用的库也在这里说一下：
* jieba 结巴中文分词，就是吧文章拆分成一个个词语
* wordcloud 词云生成器
* matplotlib 图表绘制库
* numpy python数学运算库（这个其实就是做了个计数器～）

都是非常常用的库，对 Python 和数据分析有兴趣的朋友可以照着这个表看看。（操作系统不熟Windows也可以）
对AO3的爬取并不复杂，但还是有一些难度的。经过我一系列的测试发现,Request并不能有效的爬取AO3的信息，因此就选择了selenium。对于Selenium的介绍，我就过多不多描述了，有很多技术文介绍。大家可以自行搜索。我这里给大家一些关键词，方便大家搜索。

* selenium配置chromedriver
* selenium元素定位方法

In [1]:
import sys
import re
import os
import time

from tqdm import tqdm

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

import random

爬取AO3的中文文章其实并不复杂，

 1. 我们进入AO3首页，点击 “Search” 然后点击 “Edit Your Search” 进入高级搜索模式
 2. 在Language里选择中文，点击“Search”
 3. 把滚动条拉倒最下面，点第二页
 4. 把浏览起里面的地址复制下来

 我们仔细查看这个url请求，发现这个请求的参数还是非常清晰的，让我们来看看works/后面的参数：
 
 `search?commit=Search&page=`后面跟着一个数字2，我们点击其第三页，这个数字也变成了3。所以可以断定这个参数指的是页码  
 `&work_search%5Blanguage_id%5D=` 后面跟着zh字样，可以判这个参数是控制语言的  
 同理`&work_search%5Brating_ids%5D=`控制的是分级  
 
 其他字段也是类似的，大家有兴趣可以自己试验，我就不再叙述了，值得一提的是，我在爬取时并没有用到分级标签功能，只是在搜索里面翻文章。通过多次搜索我发现AO3的搜索结果有一定的随机性，并没有主动干预搜索结果，这一点还是很良心的。

In [2]:
#获谷歌取搜索页面
def make_google_search_url(page=0, num=100):
    base_loc = 'https://www.google.com/search?hl=en&q=site:lofter.com+ao3&safe=images'
    base_loc += "&num="+str(num)
    base_loc += "&start="+str(page*num) #搜索页

    return base_loc

In [3]:
#从谷歌获取文章链接
def get_url_from_search(html):
    old_list = []
    soup = BeautifulSoup(html, 'html.parser')
    search_div = soup.find('div', attrs={'id': 'search'})
    div_g_groups = search_div.findAll('div', attrs={'class': 'g'})
    for g in div_g_groups:
        div_r = g.find('div', attrs={'class': 'r'})
        a_hurl = div_r.find('a')
        old_list.append(a_hurl['href'])
    
    return old_list

In [4]:
def find_ao3_from_lofter(lofter_url_list, browser, path):
    for url in lofter_url_list:
        print(url)
        dir_name = url.replace("http://",'').replace(".com/",'_').replace("/","_").replace(".","_")
        dir_path = os.path.join(path,dir_name)
        isExists=os.path.exists(dir_path)
        if isExists:
            print('Exists')
            continue
        # 判断结果
        ao3_links = []
        browser.get(url)
        currurl = browser.current_url
        if "archiveofourown" in currurl and ("/works/" in currurl or "/users/" in currurl): #如果url 直接跳转
            ao3_links.append(currurl)
            lhtml = ""
        else: #没有跳转
            lhtml = browser.page_source
            soup = BeautifulSoup(lhtml, 'html.parser')
            alink_groups = soup.findAll('a', attrs={'rel': 'nofollow'})
            for alink in alink_groups:
                href_str = alink['href']
                if "archiveofourown" in href_str and ("/works/" in href_str or "/users/" in href_str):
                    ao3_links.append(href_str)
        
        if ao3_links:
            # 判断路径是否存在
            isExists=os.path.exists(dir_path)

            # 如果不存在则创建目录
            os.makedirs(dir_path)

            links_str = url + '\n'
            
            need_agree = True
            for work_url in ao3_links: #遍历ao3链接
                links_str += work_url + '\n'
                
            print(os.path.join(dir_path,"links.txt"))  
            fh = open(os.path.join(dir_path,"links.txt"), 'w') #保存页面
            fh.write(links_str) #写入内容
            fh.close() #关闭
            
            print(os.path.join(dir_path,"lofter.html"))  
            fh = open(os.path.join(dir_path,"lofter.html"), 'w') #保存页面
            fh.write(lhtml) #写入内容
            fh.close() #关闭
            for work_url in ao3_links:
                browser.get(work_url)

                if need_agree:
                    try:
                        time.sleep(3)
                        browser.find_element_by_id('tos_agree').click()
                        time.sleep(1)
                        browser.find_element_by_id('accept_tos').click()
                        time.sleep(1)

                        need_agree = False
                    except NoSuchElementException:
                        need_agree = False

                work_html_text = browser.page_source  #获得页面代码
                
                work_name = work_url.replace('https://','').replace('http://','').replace('archiveofourown','').replace('.com/','').replace('.org/','').replace('/','_').replace(".","_").replace('#','_')
                work_path = os.path.join(dir_path, work_name+".html")
                
                if "If you accept cookies from our site and you choose \"Proceed\"" in work_html_text: #无法获取正文则点击Proceed
                    browser.find_element_by_link_text('Proceed').click()
                    time.sleep(1)
                    browser.get(work_url)
                    work_html_text = browser.page_source
                if "Retry later" in work_html_text:
                    while "Retry later" in work_html_text:
                        print(work_path)
                        fh = open(work_path, 'w') #保存页面
                        fh.write("Need_to_reload") #写入内容
                        fh.close() #关闭
                        print("Retry Later")
                        time.sleep(3)
                        browser.get("http://www.baidu.com")
                        time.sleep(3)
                        browser.quit()
                        c_service.stop()
                        time.sleep(60)
                        c_service.start()
                        browser = webdriver.Chrome(chrome_options=chrome_options)  #调用Chrome浏览器
                        browser.get("https://archiveofourown.org/")
                        time.sleep(5)
                        browser.find_element_by_id('tos_agree').click()
                        time.sleep(2)
                        browser.find_element_by_id('accept_tos').click()
                        time.sleep(3)

                        browser.get(work_url)

                        work_html_text = browser.page_source  #获得页面代码

                        if "If you accept cookies from our site and you choose \"Proceed\"" in work_html_text: #无法获取正文则点击Proceed
                            browser.find_element_by_link_text('Proceed').click()
                            time.sleep(1)
                            browser.get(work_url)
                            work_html_text = browser.page_source


                #if "<!--chapter content-->" in work_html_text:
                print(work_path)
                fh = open(work_path, 'w') #保存页面
                fh.write(work_html_text) #写入内容
                fh.close() #关闭
                time.sleep(float(random.randint(10,50))/10) #随机延时
    return browser
                    

In [5]:
start_p = 0
end_p = 4

In [6]:
c_service = webdriver.chrome.service.Service('/usr/bin/chromedriver')
c_service.command_line_args()
c_service.start()

chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--proxy-server=socks5://localhost:1080')

auto_quit_cnt = 0
browser = webdriver.Chrome(chrome_options=chrome_options)  # 调用Chrome浏览器
for page in range(start_p, end_p):
        print('-'*30)
        print("Page: " + str(page))
        print('-'*30)
        google_search_url = make_google_search_url(page)
        browser.get(google_search_url)
        
        html_text = browser.page_source  # 获得页面代码
        while 'Our systems have detected unusual traffic' in html_text:
            print("Google Robot!")
            time.sleep(10)
            html_text = browser.page_source  # 获得页面代码
            auto_quit_cnt += 1
            if auto_quit_cnt>30:
                break
        auto_quit_cnt = 0   
        lofter_list = get_url_from_search(html_text)
        browser = find_ao3_from_lofter(lofter_list, browser, "lofter")

/home/zhiwei/miniconda3/envs/dslen/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


------------------------------
Page: 0
------------------------------
http://niccoting.lofter.com/nicco_ting
Exists
http://kesing1219.lofter.com/ao3333
Exists
http://variouscake.lofter.com/aoaoao
Exists
http://kaoru920.lofter.com/toAO3
Exists
http://riyueshier.lofter.com/ao3account
Exists
http://iveylautv.lofter.com/achieveofourhome
Exists
http://isis0502.lofter.com/ao3333
Exists
http://smokedshark.lofter.com/ao3333
Exists
http://chilicap.lofter.com/chilicaao3
Exists
http://cabbagerabbit010.lofter.com/AO3fics
Exists
http://bitterain.lofter.com/junegardenia
Exists
http://ronanjun.lofter.com/aoao3
Exists
http://lovetitle-hq.lofter.com/23fevb
http://ghotisky.lofter.com/AO3ghotisky
Exists
http://vanuatu.lofter.com/00004
Exists
http://blurryyou.lofter.com/translations
Exists
http://jshhan.lofter.com/others
Exists
http://wuhsing.lofter.com/myao3
Exists
http://tantantangent.lofter.com/amberjune
Exists
http://blandbilker.lofter.com/mywebsite
http://redbutton.lofter.com/archiveofourown
Exists
h

http://sarriasmg.lofter.com/tag/ao3?page=3&t=1457498642408
Exists
https://lllmpf.lofter.com/post/1cd4c189_106d8f74
Exists
http://funfunf.lofter.com/tag/AO3
Exists
http://asesese.lofter.com/tag/AO3
http://shootfandom.lofter.com/post/3d7cc2_b982c8a
http://zwaryf.lofter.com/tag/AO3
Exists
http://karryrene.lofter.com/post/1d0fc85f_1c66a2311
http://destielland.lofter.com/post/1d2a6959_963ec00
http://saltandpeppernoise.lofter.com/post/1ff9f6ec_12e12fb2a
http://snowxat.lofter.com/tag/ao3
http://sunny04.lofter.com/post/1d2e1947_12d52d01c
http://qcjjhsmjcn.lofter.com/tag/AO3
Exists
http://artzeoyoyo.lofter.com/post/1ef6ab7b_1c5d5c0d1
http://navyteoh.lofter.com/post/1ff85660_12e90f37a
Exists
http://kesing1219.lofter.com/post/1dcf07fb_12c86ec29
Exists
http://abevery.lofter.com/tag/AO3
Exists
https://larrialllee.lofter.com/tag/AO3
http://qihong9475.lofter.com/tag/ao3
http://duanxiaobuzhi.lofter.com/post/1d17f638_116eafd3
https://stucky1226.lofter.com/post/1e1c1114_12d1bc3e6
Exists
http://amerland1

http://singleness.lofter.com/archiveofourown
Exists
http://evenight.lofter.com/ao000
Exists
http://kinlochhold.lofter.com/AO3AO3
Exists
http://anythingother.lofter.com/Ao3article
Exists
https://shiki1991.lofter.com/1234567
Exists
http://saviel.lofter.com/aothree
Exists
http://mi-zhuang.lofter.com/myAO3
Exists
https://nova2837.lofter.com/mywebsite2
Exists
https://yuukee.lofter.com/yukee
Exists
http://dmrwfuckyeah.lofter.com/ao333
Exists
http://pennyfs.lofter.com/pennyfsao3
http://nekropolis.lofter.com/tremella
Exists
http://minwoo1979.lofter.com/shishidoao3
Exists
https://xiaohoney.lofter.com/aooooo
Exists
http://brynhild-chen.lofter.com/mauvestingers
Exists
http://dawningli.lofter.com/myAO3
Exists
http://virgilz.lofter.com/myao3
Exists
http://scientistr.lofter.com/sciao3link
Exists
http://rastar.lofter.com/AO3-rastar
http://1421291696.lofter.com/post/1d73ee6f_10ca8c0d
http://zgkm181.lofter.com/post/1d7454f3_1138ff9a
Exists
http://lizzysun.lofter.com/post/1cace57e_1c7c244a4
Exists
http:

In [7]:
#获取文章链接
def get_work_id_from_search(html):
    old_list = []
    soup = BeautifulSoup(html, 'html.parser')
    ol = soup.find('ol', attrs={'class': 'work index group'})
    work_blurb_groups = ol.findAll('li', attrs={'class': 'work blurb group'})
    for wbg in work_blurb_groups:
        if wbg["id"] not in old_list:
            old_list.append(wbg["id"])
    
    return old_list

做好这些准备工作下面我们就开始正式爬取数据

具体思路是这样的：
1. 打开一个浏览器，需要注意的我这里使用了代理，否则无法浏览到AO3；
2. 通过selenium `find_element_by_id` 功能找到相应按钮自动点击，同意网站条款；
3. 进入循环，通过`make_search_url`函数组合出搜索页的链接，遍历页码；
4. 将搜索页的html传入函数`get_work_id_from_search`提取出所有文章id；
5. 遍历文章id通过文章id组合出文章地址并访问，最后保存文章页面的html。

这其中有两个注意事项：
1. 当进入限制级文章时，网站会提示再次同意浏览条款，当检测到条款关键字时，使用`find_element_by_link_text('Proceed').click()`点击确认即可；
2. 频繁访问后，网站会拒绝访问请求出现‘Retry later’页面，当检测到这种情况后，进行异常处理，关闭当前的浏览器，等待一分钟后重新访问。（这也是爬取文章速度比较慢的原因，有大神知道怎么解决的请赐教）

这里声明一下，我是不会教大家如何挂代理的，这个本身就是我自己专用的，私下要也不会给的。